In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('real_estate_value.csv')

In [3]:
df.dtypes

HouseAge         float64
DistanceToMRT    float64
NoOfStores         int64
Latitude         float64
Longitude        float64
UnitPrice        float64
dtype: object

In [4]:
df = df.dropna()

In [5]:
y = df['UnitPrice']
x = df.drop('UnitPrice', axis=1)

In [6]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
symmetric = []
skewed = []

for column in x.columns:
    if df[column].skew() > -1 and df[column].skew() < 1:
        symmetric.append(column)
    else:
        skewed.append(column)

In [8]:
symmetric_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('scaler', StandardScaler())
])

skewed_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('log_transform', FunctionTransformer(np.log1p)),
  ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer([
    ('numeric_symmetric', symmetric_pipeline, symmetric),
    ('numeric_skewed', skewed_pipeline, skewed),
])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [9]:
pipeline.fit(df)
train_transformed = pipeline.transform(train_x)
test_transformed = pipeline.transform(test_x)

In [10]:
decision_tree = DecisionTreeRegressor(random_state=42)
decision_tree.fit(train_transformed, train_y)
pred_dt = decision_tree.predict(test_transformed)

In [11]:
mse_dt = mean_squared_error(test_y, pred_dt)
r2_dt = r2_score(test_y, pred_dt)

In [12]:
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(train_transformed, train_y)
pred_rf = random_forest.predict(test_transformed)

In [13]:
mse_rf = mean_squared_error(test_y, pred_rf)
r2_rf = r2_score(test_y, pred_rf)

In [14]:
print("Mean Squared Error (Decision Tree):", mse_dt)
print("R-squared (Decision Tree):", r2_dt)
print("Mean Squared Error (Random Forest):", mse_rf)
print("R-squared (Random Forest):", r2_rf)

Mean Squared Error (Decision Tree): 55.57156626506025
R-squared (Decision Tree): 0.6687431396370993
Mean Squared Error (Random Forest): 34.83787655745048
R-squared (Random Forest): 0.7923347066539833
